<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 


In [6]:
/* Create External File Format */

USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS(
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE)
    );
END

Commands completed successfully.

Total execution time: 00:00:00.0818304

Commands completed successfully.

Total execution time: 00:00:00.4794854

In [7]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-master-pool:50070');
END

Commands completed successfully.

Total execution time: 00:00:00.3879082

In [8]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'partner_customers_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [partner_customers_hdfs]
    ("CustomerSource" VARCHAR(250) 
    , "CustomerName" VARCHAR(250) 
    , "EmailAddress" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.4509022

In [9]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP 10 CustomerSource
, CustomerName
, EMailAddress
    FROM [partner_customers_hdfs] hdfs
WHERE EmailAddress LIKE '%wingtip%';
GO


: Msg 8680, Level 17, State 87, Line 3
Internal Query Processor Error: The query processor encountered an unexpected error during the processing of a remote query phase.

Total execution time: 00:00:00.2393277

In [10]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */

SELECT TOP 10 a.FullName, b.CustomerSource
  FROM [WideWorldImporters].[Application].[People] a
  INNER JOIN [WideWorldImporters].[dbo].[partner_customers_hdfs] b  ON a.EmailAddress = b.EmailAddress;
  GO

: Msg 8680, Level 17, State 87, Line 4
Internal Query Processor Error: The query processor encountered an unexpected error during the processing of a remote query phase.

Total execution time: 00:00:00.2139328

## Next Steps: Continue on to Working with the SQL Server Data Pool